In [ ]:
import pandas

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

with open('data/nt_full_060124.html', 'r', encoding='utf-8') as f:

    html_content = f.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract rows
    table_content = soup.select_one('tbody.table-content')
    # Extracting data for each player
data = []
for row in table_content.select('tr.hasCaptainsCol'):

    name = row.select_one('td.name-cell span').text
    position = row.select_one('td.position-cell ft-tooltip')['text']
    team = row.select_one('td.team-cell ft-tooltip')['text']
    price = row.select_one('td.price-cell').text
    ownership_pct = row.select_one('td.selectedBy-cell span.value').text
    captain_pct = row.select_one('td.captainedBy-cell span.value').text
    points = row.select_one('td.points-cell ft-bubble span').text
    
    data.append([name, position, team, price, ownership_pct, captain_pct, points])

    
# Convert data to a pandas DataFrame
norsk_tipping = pd.DataFrame(data)

print(norsk_tipping)


In [ ]:
norsk_tipping.columns = ["Name", "Position", "Team", "Price", "Ownership", "Captain%", "PointsRound"]

In [ ]:
norsk_tipping["Price"] = norsk_tipping["Price"].str.extract("^[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)M")[0]
norsk_tipping["Price"] = pd.to_numeric(norsk_tipping["Price"])

In [ ]:
norsk_tipping.sort_values("Price", inplace=True)

In [ ]:
norsk_tipping["Surname"] = norsk_tipping["Name"].str.rsplit(". ", n=1, expand=True).fillna("").apply(lambda x : max(x,key=len),axis=1)

In [ ]:
norsk_tipping

## Get values from FPL Review

In [ ]:
fplreview = pd.read_csv("confidential_data/fplreview_1704915028.csv")

point_columns = [ col for col in fplreview.columns if "_Pts" in col]

point_columns

fplreview["Total Points"] = fplreview[point_columns].sum(axis=1)

fplreview.sort_values("Total Points",ascending=False, inplace=True)
fplreview

In [ ]:
norsk_tipping["Surname"].isin(fplreview["Name"].to_list()).value_counts()

In [ ]:
from unidecode import unidecode

fplreview["Name"] = fplreview["Name"].str.rsplit(".", n=1, expand=True).fillna("").apply(lambda x : max(x,key=len),axis=1).apply(unidecode)
norsk_tipping["Surname"] = norsk_tipping["Surname"].apply(unidecode).str.strip()

In [ ]:
mapper = {"Man City": "Manchester City",
          "Man Utd": "Manchester United",
          "Nott'm Forest": "Nottingham Forest",
          "Sheffield Utd": "Sheffield United",
         }

In [ ]:
fplreview["Team"] = fplreview.Team.replace(mapper)

## Replace any name that is mismatched

In [ ]:
liverpool = {"Nunez": "Darwin", 
             "Jota": "Diogo J", 
             "Diaz": "Luis Diaz", 
             "Alisson": "Becker", 
             "van Dijk": "Virgil", 
             "Alcantara": "Thiago"}

man_city = {"Silva": "Bernardo", 
            "Rodri":"Rodrigo", 
            "Ederson": "Ederson M", 
            "Dias": "Ruben",
            "Ortega": "Ortega Moreno",
            "Gomez": "Sergio Gomez",
            "Nunes": "Matheus N"}

aston_villa = {"Luiz": "Douglas Luiz",
                "Traore": "Bertrand Traore",
                "Torres": "Pau",
                "Moreno": "Alex Moreno",
                "Carlos": "Diego Carlos"}

wolves = {"Hee-chan": "Hee Chan",
          "Silva": "Fabio Silva",
          "Ait Nouri": "Ait-Nouri",
          'Sa': "Jose Sa",
          'Antonio Gomes': "Toti",
          #'H.Bueno',
          #'S.Bueno',
          'Traore': "Boubacar Traore",
          #'Sanderson': " No entry",
          'Castro': "Otto",
          'Gomes': "Joao Gomes"}

fulham = { 'Adarabioyo': "Tosin", 
          'De-Cordova Reid': "De Cordova-Reid", 
          'Traore': "Adama", 
          'Jimenez': "Raul", 
          'Pereira': "Andreas"}

brighton = {' van Hecke': "Van Hecke" , 
            #'Sima':"No entry", 
            'Pedro': "Joao Pedro", 
            'Fati': "Ansu Fati"}

spurs = {
         'Emerson': "Royal", 
          'van de Ven': "Van de Ven", 
          'Gil': "Bryan", 
          'Porro': "Pedro Porro", 
          'Bentacur': "Bentancur"
}

chelsea = {
    'Fernandez': "Enzo", 
    'Washington': "Deivid"
}

everton = {
    'Gomes': "Andre Gomes", 
    'Gueye': "Gana", 
    #'Chermiti': "Not added (yet?)", 
    'Alli': "Dele"}

arsenal = {
    "Vieira": "Fabio Vieira",
    "Partey": "Thomas",
    "Soares": "Cedric"}

crystal_palace = {
    'Ferguson': "Nathan Ferguson", 
    'Franca': "Matheus Franca"}

unique = {"Guimaraes": "Bruno G",
          'van de Beek': "Van de Beek", 
          'Bayindir': "Altay", 
          'Mejbri': "Hannibal",
          "Hedilazio": "Benson",
          
          }

In [ ]:
norsk_tipping.loc[norsk_tipping.Team=="Liverpool", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Liverpool", "Surname"].replace(liverpool)
norsk_tipping.loc[norsk_tipping.Team=="Manchester City","Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Manchester City","Surname"].replace(man_city)
norsk_tipping.loc[norsk_tipping.Team=="Aston Villa", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Aston Villa", "Surname"].replace(aston_villa)
norsk_tipping.loc[norsk_tipping.Team=="Wolves", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Wolves", "Surname"].replace(wolves)
norsk_tipping.loc[norsk_tipping.Team=="Fulham", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Fulham", "Surname"].replace(fulham)
norsk_tipping.loc[norsk_tipping.Team=="Brighton", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Brighton", "Surname"].replace(brighton)
norsk_tipping.loc[norsk_tipping.Team=="Spurs", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Spurs", "Surname"].replace(spurs)
norsk_tipping.loc[norsk_tipping.Team=="Chelsea", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Chelsea", "Surname"].replace(chelsea)
norsk_tipping.loc[norsk_tipping.Team=="Everton", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Everton", "Surname"].replace(everton)
norsk_tipping.loc[norsk_tipping.Team=="Arsenal", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Arsenal", "Surname"].replace(arsenal)
norsk_tipping.loc[norsk_tipping.Team=="Crystal Palace", "Surname"] = norsk_tipping.loc[norsk_tipping.Team=="Crystal Palace", "Surname"].replace(crystal_palace)

norsk_tipping["Surname"] = norsk_tipping["Surname"].replace(unique)

In [ ]:
df = norsk_tipping.merge(fplreview,left_on=["Team", "Surname"], right_on=["Team", "Name"], how="left",suffixes=("_nt", "_review"))

In [ ]:
df[df.Pos.isna()].Team.value_counts()
                

In [ ]:
df[(df.Pos.isna()) & (df.Team == "Brentford")]#.Surname.to_list()
                

In [ ]:
fplreview[fplreview.Team.isin(["Brentford"])].head(50).sort_values("Name")

In [ ]:
df.sort_values("Total Points",ascending=False).head(50)
                

# Top 50 GK

In [ ]:
df[df.Position == "Keeper"].sort_values("Total Points",ascending=False).head(50)

# Top 50 Defence

In [ ]:
df[df.Position == "Forsvar"].sort_values("Total Points",ascending=False).head(50)

# Top 50 Midfield

In [ ]:
df[df.Position == "Midtbane"].sort_values("Total Points",ascending=False).head(50)

# Top 50 Forwards

In [ ]:
df[df.Position == "Angrep"].sort_values("Total Points",ascending=False).head(50)

# Calculate Points-Per-Million (PPM)

In [ ]:
df["PPM"] = df["Total Points"]/(df["Price"]*12)

# Value Top 50

In [ ]:
df.sort_values("PPM", ascending=False).head(50)

# Value Top 50 GK

In [ ]:
df[df.Position == "Keeper"].sort_values("PPM",ascending=False).head(50)

# Value Top 50 Defence

In [ ]:
df[df.Position == "Forsvar"].sort_values("PPM",ascending=False).head(50)

# Value Top 50 Midfield

In [ ]:
df[df.Position == "Midtbane"].sort_values("PPM",ascending=False).head(50)

# Value Top 50 Forwards

In [ ]:
df[df.Position == "Angrep"].sort_values("PPM",ascending=False).head(50)

# Out of position

In [ ]:
position_mapper = { "G": "Keeper", "D": "Forsvar", "M": "Midtbane", "F": "Angrep"}

In [ ]:
df[df.Pos.map(position_mapper) != df.Position].dropna()

There is clearly no outstanding picks, and Davies at Sheffield United is mismatched, need a way to figure out duplicates.